In [1]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta
import sqlalchemy
import urllib

from TAPI import capturar_fecha

#funciones

def iterar_entre_fechas(fecha_desde, fecha_hasta):
    fecha_actual = datetime.strptime(fecha_desde, "%Y-%m-%dT%H:%M:%S.%f%z")
    fecha_fin = datetime.strptime(fecha_hasta, "%Y-%m-%dT%H:%M:%S.%f%z")

    # Asegurarse de que fecha_actual sea exactamente a la medianoche
    fecha_actual = fecha_actual.replace(hour=0, minute=0, second=0, microsecond=0)

    while fecha_actual <= fecha_fin:
        fecha_siguiente = fecha_actual + timedelta(hours=23, minutes=59)
        yield fecha_actual, fecha_siguiente
        # Añadir un día para la próxima iteración
        fecha_actual += timedelta(days=1)

2024-05-06


In [2]:
fecha_desde = "2023-01-01T00:00:00.000-03:00"
fecha_hasta = "2024-05-05T23:59:59.000-03:00"

for fecha_actual, fecha_siguiente in iterar_entre_fechas(fecha_desde, fecha_hasta):
    dia_mdb = fecha_actual.strftime("%d-%m-%Y")

In [2]:
# DATOS DE LA BBDD, SERVER Y TABLA

#server = 'DESKTOP-37ESKFT\SQLEXPRESS'
server = 'DARCCVWSQL19'
database = 'TAPI'

#1
#-------------------------------------------------------#
#tabla = 'DiarioTest'
#tabla = "Hist_Bk"
#tabla = 'Contratos_diario'
tabla = "Contratos_Corregidos_bk"
#-------------------------------------------------------#

#2
#-------------------------------------------------------#
# Fechas para seleccionar el día de la carga se debe iterar

fecha_desde = "2023-01-01T00:00:00.000-03:00"
fecha_hasta = "2024-03-31T23:59:59.000-03:00"

#Defino la tabla de CAMMESA que me voy a traer
NEMO = "PARTE_POST_OPERATIVO_UNIF"
#NEMO = "PARTE_POST_OPERATIVO"
#-------------------------------------------------------#
#URL para capturar Id del documento y el zip file:

#Busco los zip disponibles para traer así puedo extraer el id
URL = f"https://api.cammesa.com/pub-svc/public/"

method_id = "findDocumentosByNemoRango?" #ID
method_zip = "findAllAttachmentZipByNemoId?"


zip_path = r"C:\Users\jadurian\Documents\Tapi\.zips"
mdb_path = r"C:\Users\jadurian\Documents\Tapi\.zips\.mdb"

connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Establecer la conexión con la base de datos de SQL Server
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

#url_doc_id = f"{URL}{method_id}fechadesde={fecha_desde}&fechahasta={fecha_hasta}&nemo={NEMO}"

dataframes = []
dfout = pd.DataFrame()
df_filtrado = pd.DataFrame()

for fecha_actual, fecha_siguiente in iterar_entre_fechas(fecha_desde, fecha_hasta):
    valores_generadores = pd.DataFrame()
    contrato_abastecimiento = pd.DataFrame()

    url_doc_id = f"{URL}{method_id}fechadesde={fecha_actual.isoformat()}&fechahasta={fecha_siguiente.isoformat()}&nemo={NEMO}"
    
    #obtener el doc_id del dia actual (corregido)
    dia_mdb = fecha_actual.strftime("%d-%m-%Y")
    try:
        with requests.get(url_doc_id) as response:
            if response.status_code == 200:
                PPO=response.json()
                doc_id = PPO[-1]['id']
                print(doc_id)
            else:
                print("La solicitud falló con el código de estado:", response.status_code)
    except requests.exceptions.RequestException as e:
        # Manejar la excepción
        print("Error al realizar la solicitud:", e)

    url_zip = f"{URL}{method_zip}docId={doc_id}&nemo={NEMO}"

    #descargar el .zip del doc_id (corregido)

    try:
        with requests.get(url_doc_id) as response:
            if response.status_code == 200:
                r = requests.get(url_zip)

                # Crear un objeto ZipFile a partir del contenido descargado
                z = zipfile.ZipFile(io.BytesIO(r.content))

                # Directorio de destino para extraer los archivos ZIP
                destination_directory = ".zips"

                # Extraer todos los archivos del ZIP en el directorio específico
                z.extractall(destination_directory)
                zip_name = z.namelist()[0]
            else:
                print("La solicitud falló con el código de estado:", response.status_code)

    except requests.exceptions.RequestException as e:
        # Manejar la excepción
        print("Error al realizar la solicitud:", e)
    
    #Colocar los PATHs correctos donde se traeran los archivos
    
    path_zip_dia = f"{zip_path}\{zip_name}"

    #display(path_zip_dia)

    
    try:
        # Extrae el archivo MDB de cada archivo ZIP diario
        with zipfile.ZipFile(path_zip_dia, 'r') as zip_ref:
            # Encontrar el nombre del archivo MDB dentro del ZIP diario
            archivo_mdb = os.path.splitext(zip_name)[0] + ".mdb"
            zip_ref.extract(archivo_mdb, path=mdb_path)

   
        # Lee el archivo MDB y cargar la tabla VALORES_GENERADORES en un dataframe
        mdb_file = os.path.join(mdb_path, archivo_mdb)
        conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={mdb_file};"
        conn = pyodbc.connect(conn_str)

        #3
        #-------------------------------------------------------#
        #valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)
        contrato_abastecimiento = pd.read_sql("SELECT * FROM CONTRATO_ABASTECIMIENTO", conn)
        #-------------------------------------------------------#
        conn.close()
        
        # Convertir dia_mdb a un objeto datetime
        dia_datetime = datetime.strptime(dia_mdb, '%d-%m-%Y')

        # Formatear la fecha en el formato YYYY-MM-DD como una cadena
        dia_mdb_formatted = dia_datetime.strftime('%Y-%m-%d')

        # Insertar la fecha formateada en la lista valores_generadores
        
        #4
        #-------------------------------------------------------#
        #valores_generadores.insert(0, 'FECHA', dia_mdb_formatted)

        contrato_abastecimiento.insert(0, 'FECHA', dia_mdb_formatted)
        #-------------------------------------------------------#
        quoted = urllib.parse.quote_plus(connection_string)

        #Por limitaciones de tamaño de excel filtramos solo las máquinas Pampa
        valores_filtrados = ["ADTOHI", "AR21EO", "BAHIEO", "BBLATV29", "BBLATV30",
                            "BBLMDI01", "BBLMDI02", "BBLMDI03", "BBLMDI04", 
                            "BBLMDI05", "BBLMDI06", "CERITV01", "CORTEO", 
                            "EBARTG01", "EBARTG02", "EBARTV01", "ETIGHI", 
                            "GEBATG01", "GEBATG02", "GEBATG03", "GEBATG04", 
                            "GEBATV01", "GEBATV02", "GUEMTG01", "GUEMTV11", 
                            "GUEMTV12", "GUEMTV13", "LDLATG01", "LDLATG02", 
                            "LDLATG03", "LDLATG04", "LDLATG05", "LDLATV01", 
                            "LDLMDI01", "LREYHB", "NIH1HI", "NIH2HI", "NIH3HI", 
                            "PAMEEO", "PEP3EO", "PILBDI01", "PILBDI02", 
                            "PILBDI03", "PILBDI04", "PILBDI05", "PILBDI06", "PIQIDI01", "PPLEHI"]
        
        contratos_filtrados = ["C.T. LOMA DE LA LATA", "C.T.E.BARRAGAN TV-M", "CT LOMA II LA LATA-M", "GENELBA CC -MERCA", "PIEDRABUENA  R21-","CT PILAR BS AS M"]

        # Filtrar el DataFrame por los valores especificados en la columna "GRUPO"

        #DEFINIR EL DF A SUBIR

        #5
        #-------------------------------------------------------#

        #df_filtrado = valores_generadores[valores_generadores["GRUPO"].isin(valores_filtrados)]  
        
        df_contratos = contrato_abastecimiento[contrato_abastecimiento["CONTRATO"].isin(contratos_filtrados)] 

        #-------------------------------------------------------#

        #TODO: Revisar el filtrado previo a igresar la data    
        
        #df_filtrado = df_filtrado.sort_values(by=['FECHA', 'HORA', 'GRUPO'])

        engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
    
        #ACÁ ESCRIBE LA DATA, REVISAR SIEMPRE ARGUMENTOS

        #6
        #-------------------------------------------------------#

        #df_filtrado.to_sql(f'{tabla}', schema='dbo', con=engine, if_exists='append', chunksize=20000)

        df_contratos.to_sql(f'{tabla}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
        #-------------------------------------------------------#


        #dfout = pd.concat([dfout,df_filtrado], ignore_index=True)

        #dfout.to_sql(f'{tabla}', schema='dbo', con=engine, if_exists="replace", chunksize=20000)

    except FileNotFoundError:
        print(f"El archivo {zip_name} no se encontró. Saltando al siguiente archivo...")

01255BEF763F7DD003258951005A1698


C:\Users\jadurian\AppData\Local\Temp\ipykernel_18992\1678659200.py:118: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  contrato_abastecimiento = pd.read_sql("SELECT * FROM CONTRATO_ABASTECIMIENTO", conn)


942AA61382F54C3003258951005A7DC5


C:\Users\jadurian\AppData\Local\Temp\ipykernel_18992\1678659200.py:118: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  contrato_abastecimiento = pd.read_sql("SELECT * FROM CONTRATO_ABASTECIMIENTO", conn)


65A52C294183E45903258951005A8D9D


...

C:\Users\jadurian\AppData\Local\Temp\ipykernel_18992\1678659200.py:118: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  contrato_abastecimiento = pd.read_sql("SELECT * FROM CONTRATO_ABASTECIMIENTO", conn)


8703BCF9C5D59A260325896E00432D10


C:\Users\jadurian\AppData\Local\Temp\ipykernel_18992\1678659200.py:118: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  contrato_abastecimiento = pd.read_sql("SELECT * FROM CONTRATO_ABASTECIMIENTO", conn)


F9086FEAA3A7F2140325898E005AAC24


BadZipFile: File is not a zip file

In [19]:
valores_generadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15984 entries, 0 to 15983
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   FECHA                15984 non-null  object 
 1   GRUPO                15984 non-null  object 
 2   HORA                 15984 non-null  int64  
 3   ARL                  15984 non-null  int64  
 4   ESTADO_OPE           15984 non-null  object 
 5   PI_SOTR              15984 non-null  float64
 6   ARL_ECON             15984 non-null  int64  
 7   EST_REMUN            15984 non-null  object 
 8   ENERGIA              15984 non-null  float64
 9   PRECIO_NODO          15984 non-null  float64
 10  PR_REM_ENERGIA       9806 non-null   float64
 11  SMEC                 15984 non-null  object 
 12  POT_DISP             15984 non-null  float64
 13  POT_OPE_CPO          13536 non-null  float64
 14  COMPRA_SPOT          15168 non-null  float64
 15  SCTD                 9249 non-null  

In [20]:
quoted = urllib.parse.quote_plus(connection_string)
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
dfout.to_sql('DiarioTest', schema='dbo', con=engine, if_exists='append')

15